## 線上資源
- Generate Generalized Amber Force Field (gaff) ([link](https://ambermd.org/tutorials/basic/tutorial4b/))
- Step by Step Youtube Video and Github Source ([youtube](https://www.youtube.com/watch?v=c_EMS1RVDxQ)) ([github](https://github.com/huichenggong/Learning-Computation-with-Chenggong/tree/main/1-MD_tutorial/1-5-Antechamber-GAFF-non-standard-residue-prepare))
- Amber 2023 mounal ([link](https://ambermd.org/doc12/Amber23.pdf))

In [1]:
drug = "POH"

### 從PDB資料庫中下載藥物分子結構(sdf)

In [2]:
!wget https://files.rcsb.org/ligands/download/{drug}_ideal.sdf

--2025-03-31 15:14:08--  https://files.rcsb.org/ligands/download/POH_ideal.sdf
Resolving files.rcsb.org (files.rcsb.org)... 3.169.36.44, 3.169.36.56, 3.169.36.123, ...
Connecting to files.rcsb.org (files.rcsb.org)|3.169.36.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6422 (6.3K) [application/octet-stream]
Saving to: ‘POH_ideal.sdf.1’

POH_ideal.sdf.1     100%[===================>]   6.27K  --.-KB/s    in 0s      

2025-03-31 15:14:08 (105 MB/s) - ‘POH_ideal.sdf.1’ saved [6422/6422]



### 從 .sdf 檔轉成 .gjf 檔
1. Open GaussView (6.0.16)
2. Click File > Open... > {drug}_ideal.sdf
3. Click File > Save > File name: {drug}.gjf, File of type: *.gjf
4. 手動調整gjf檔
nproc: 使用32核心平行計算<br>
mem   : 使用20GB的記憶體進行計算<br>
chk   : 建立{drug}.chk作為checkpoint file, 若伺服器因不明原因shunt down job, 可以從中斷點開始跑<br>
opt   : 能量最小化<br>
{drug}: 分子名稱<br>
4 1   : 第一個參數為分子電荷量，此分子的形式電荷為+4。第二個參數為分子的multiplicity，一般型況下為1<br>
```
%nproc=32
%mem=20GB
%chk=POH.chk
# opt b3lyp/6-31g(d) scrf=(smd,solvent=water) nosymm pop=mk geom=connectivity iop(6/33=2,6/42=6)

POH

4 1
```

### 執行能量最小化 
AMBER有兩種層級來以能量最小化計算RESP電荷 <br>
1. HF/6-31G* 但不使用implicit solvent model <br>
2. B3LYP/6-31G* 使用implicit solvent model <br>
兩種方法都有人使用，我們使用第二種層級，因為在藥物POH中，以B3LYP計算POH的最小化結構較準確。 <br>
A. C. Maria 也用第二種層級建立硼基共價配體 ([link](https://www.mdpi.com/1420-3049/28/6/2866#B34-molecules-28-02866)) <br>
(一般花上數小時至數天不等，也可以在HF層級先做最小化後，在轉b3lyp層級，可能會比較快)

In [3]:
!g16 {drug}.gjf
## 若要在背景執行，在terminal輸入：
## nohup g16 {drug}.gjf &

### 進行原子類型 (atomtype) 的命名與原子部分電荷 (partial charge) 的分配
antechamber ([maunal](https://ambermd.org/doc12/Amber23.pdf#page=313)) <br>
-i  輸入檔案名稱 <br>
-fi 輸入檔案的格式(如若輸入檔案為.gjf即為gout，其他在maunal的P.314 List of the File Formats列表) <br>
-o  輸出檔案名稱 <br>
-fo 輸出檔案格式 <br>
-c  以RESP進行電荷分配 <br>
-rn 分子名稱 <br>
-at 原子類型 (amber: for PARM94/99/99SB) <br>
(確保conda環境可執行AmberTool軟體)

In [4]:
!antechamber -i {drug}.log -fi gout -o {drug}.mol2 -fo mol2 -c resp -rn {drug} -at amber


Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to amber; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for Gaussian Output File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Info: Bond types are assigned for valence state (10) with penalty (2).






### 建立intramolecular的力場參數(force field parameter)
parmchk2 ([maunal](https://ambermd.org/doc12/Amber23.pdf#page=316)) <br>
-i 輸入檔案名稱 <br>
-f 輸入檔案格式 <br>
-o 輸出檔案名稱 <br>
-s 使用的力場參數(在maunal P.316有可使用的列表) <br>
-a 若為Y則列出所有力場參數，而若為N則僅列出未定義(缺失)的力場參數 <br>
在輸出檔案.frcmod中，penalty score=0是有在該力場存在的參數，而penalty score>0的則是以相似的結構近似的力場參數
Be careful to those problem parameters marked by "ATTN, need revision"

In [5]:
!parmchk2 -i {drug}.mol2 -f mol2 -o {drug}.all.frcmod -s gaff -a Y

In [6]:
## 修正力場參數 (POH藥物需要轉移 Calix[4]pyrrole 力場)
## CA-CA-CD-CC 這個參數對比最佳化結構完全不合理，先給定參數=0，後續再用mdgx計算較準確的力場
import re

def replace_frcmod_parameters(new_params, inp_frcmod_file, out_frcmod_file):
    with open(inp_frcmod_file, 'r', encoding='utf-8') as f:
        frcmod_content = f.readlines()

    new_params_dict = {}
    for line in new_params.strip().split('\n'):
        key = line.split()[0]  # 提取鍵類型（如 CA-N*-CA）
        if key not in new_params_dict:
            new_params_dict[key] = []
        new_params_dict[key].append(line)  # 支持同一鍵類型對應多條參數

    updated_content = []
    for line in frcmod_content:
        match = re.match(r"^(\S+-\S+-\S+).*", line)
        if match and match.group(1) in new_params_dict:
            updated_content.extend([ item+'\n' for item in new_params_dict[match.group(1)] ])  # 若有多條參數，逐條替換
        else:
            updated_content.append(line)
            
    with open(out_frcmod_file, 'w', encoding='utf-8') as f:
        f.writelines(updated_content)

calix_4_pyrrole_ff = f'''\
C*-C*-CC   63.000     108.890   ; C*-CA-CA in calix    
C*-C*-HA   35.000     123.450   ; CA-CA-HJ in calix
C*-CC-NA   70.000     105.070   ; CA-C*-NA in calix
CC-C*-HA   35.000     127.650   ; C*-CA-HJ in calix
CC-NA-CC   70.000     112.280   ; C*-NA-C* in calix
CC-NA-H    30.000     122.430   ; C*-NA-H  in calix
CA-N*-CA   69.900     119.324   same as ca-nc-ca, penalty score=  0.0; modify theta0 by MP2
C*-CC-CD   62.170     124.958   same as cc-cc-cc, penalty score=  0.0; modify theta0 by MP2
CC-CD-CC   62.170     126.263   same as cc-cc-cc, penalty score=  0.0; modify theta0 by MP2
CD-CC-NA   67.800     125.610   same as cc-cc-na, penalty score=  0.0; modify theta0 by MP2
CD-CC-NB   67.340     125.122   same as cc-cc-nc, penalty score=  0.0; modify theta0 by MP2
C*-CC-NB   67.340     111.910   same as cc-cc-nc, penalty score=  0.0; modify theta0 by MP2
CC-NB-CC   73.100     104.347   same as cc-nc-cc, penalty score=  0.0; modify theta0 by MP2
CA-CA-CD-CC   4    0.000         0.000           2.000      ;need fit by mdgx
CA-CA-CD-CC   4    0.000         0.000           4.000      ;need fit by mdgx
'''
replace_frcmod_parameters(calix_4_pyrrole_ff, f'{drug}.all.frcmod', f'{drug}.all.modify.frcmod')

### 輸出 Amber 格式的座標檔案 (coordinate file, .rst7) 與參數檔案 (parameter file, .prmtop)

In [7]:
tleap_script = f'''\
{drug}=loadmol2 {drug}.mol2
loadamberparams {drug}.all.modify.frcmod
check ## just for check
saveamberparm {drug} {drug}.prmtop {drug}.rst7
quit
''' 

!echo "{tleap_script}" > tleap.leap
!tleap -f tleap.leap 

-I: Adding /home/user/anaconda3/envs/AmberTools23/dat/leap/prep to search path.
-I: Adding /home/user/anaconda3/envs/AmberTools23/dat/leap/lib to search path.
-I: Adding /home/user/anaconda3/envs/AmberTools23/dat/leap/parm to search path.
-I: Adding /home/user/anaconda3/envs/AmberTools23/dat/leap/cmd to search path.
-f: Source tleap.leap.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.leap
Loading Mol2 file: ./POH.mol2
Reading MOLECULE named POH
Loading parameters: ./POH.all.modify.frcmod
Reading force field modification type file (frcmod)
Reading title:
Remark line goes here
check: A maximum of two parameters are acceptable:
     a UNIT and an (optional) PARMSET.
Checking Unit.

/home/user/anaconda3/envs/AmberTools23/bin/teLeap: Warning!
The unperturbed charge of the unit (3.999998) is not zero.

/home/user/anaconda3/envs/AmberTools23/bin/teLeap: Note.
Ignoring the warning from Unit Checking.

Building topology.
Building atom parameters.
Building bond parameters.
Build

### 匯出 Gromacs 格式的座標(.gro)、拓撲與參數檔案(.top)

In [8]:
!acpype -p {drug}.prmtop -x {drug}.rst7

| ACPYPE: AnteChamber PYthon Parser interfacE v. 2023.10.27 (c) 2025 AWSdS |
Converting Amber input files to Gromacs ...
==> Writing GROMACS files

==> Disambiguating lower and uppercase atomtypes in GMX top file, even if identical.

==> Writing GMX dihedrals for GMX 4.5 and higher.

Total time of execution: less than a second
